In [1]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.5
    Uninstalling langchain-text-splitters-0.3.5:
      Successfully uninstalled langchain-text-splitters-0.3.5
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.17
    Uninstalling langchain-0.3.17:
      Successfully uninstalled langchain-0.3.17


In [2]:
# from langchain.agents import create_sql_agent
# from langchain.chat_models import ChatOpenAI
# from langchain.sql_database import SQLDatabase
# import sqlite3

from langchain.agents import create_sql_agent
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
import sqlite3

In [3]:
# Step 1: Set up a sample SQLite database
def create_sample_database():
    connection = sqlite3.connect("sample.db")
    cursor = connection.cursor()

    # Create tables
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS startups (
        id INTEGER PRIMARY KEY,
        name TEXT,
        domain TEXT,
        founding_year INTEGER
    )
    """)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS founders (
        id INTEGER PRIMARY KEY,
        name TEXT,
        startup_id INTEGER,
        FOREIGN KEY (startup_id) REFERENCES startups (id)
    )
    """)

    # Insert sample data
    cursor.execute("INSERT INTO startups (name, domain, founding_year) VALUES ('HealthAI', 'healthcare', 2021)")
    cursor.execute("INSERT INTO startups (name, domain, founding_year) VALUES ('MediTech', 'healthcare', 2019)")
    cursor.execute("INSERT INTO startups (name, domain, founding_year) VALUES ('AIWorks', 'finance', 2022)")
    cursor.execute("INSERT INTO startups (name, domain, founding_year) VALUES ('AVHealthCareAI', 'healthcare', 2024)")

    cursor.execute("INSERT INTO founders (name, startup_id) VALUES ('Alice', 1)")
    cursor.execute("INSERT INTO founders (name, startup_id) VALUES ('Bob', 2)")
    cursor.execute("INSERT INTO founders (name, startup_id) VALUES ('Charlie', 3)")
    cursor.execute("INSERT INTO founders (name, startup_id) VALUES ('Anand Vishwakarma', 4)")

    connection.commit()
    connection.close()

In [4]:
# Step 2: Initialize the LangChain SQL agent
def initialize_agent():
    # Connect to the SQLite database
    db = SQLDatabase.from_uri("sqlite:///sample.db")
    llm = ChatOpenAI(model="gpt-4", temperature=0)
    toolkit = SQLDatabaseToolkit(db=db, llm=llm)
    agent = create_sql_agent(
        llm=llm,
        toolkit=toolkit,
        verbose=True
    )
    return agent

In [5]:
# Step 3: Query the agent
def query_agent(agent, query):
    response = agent.run(query)
    print("\n### Response ###")
    print(response)

In [6]:
create_sample_database()

In [7]:
from google.colab import userdata
my_secret_key = userdata.get('OPENAI_API_KEY')

# Set OpenAI API key
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [8]:
# Initialize the multi-hop agent
agent = initialize_agent()

<ipython-input-4-1798643b86a7>:16: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4", temperature=0)


In [9]:
# Example query: Multi-hop reasoning
query = """
Which startups founded after 2020 are working in the healthcare domain,
and who are their founders?
"""

query_agent(agent, query)

<ipython-input-5-4226184f624b>:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(query)




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: ""founders, startupsThe tables "founders" and "startups" seem relevant to the question. I should check their schema to understand the structure and the fields they contain.
Action: sql_db_schema
Action Input: "founders, startups"
CREATE TABLE founders (
	id INTEGER, 
	name TEXT, 
	startup_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(startup_id) REFERENCES startups (id)
)

/*
3 rows from founders table:
id	name	startup_id
1	Alice	1
2	Bob	2
3	Charlie	3
*/


CREATE TABLE startups (
	id INTEGER, 
	name TEXT, 
	domain TEXT, 
	founding_year INTEGER, 
	PRIMARY KEY (id)
)

/*
3 rows from startups table:
id	name	domain	founding_year
1	HealthAI	healthcare	2021
2	MediTech	healthcare	2019
3	AIWorks	finance	2022
*/The "startups" table contains the "founding_year" and "domain" fields which are relevant to the question. The "founders" table contains the "name" field which represents the founder's name and "startup_